In [17]:
python run_parser_and_generate_summary.py --nccl-debug-log gpt2_rccl_mp4_log_newPR.txt --rocm --unique


2

In [ ]:
topo_matrix = [[ 0, 15, 30, 15, 72, 72, 72, 72],
               [15,  0, 15, 30, 72, 72, 72, 72],
               [30, 15,  0, 15, 72, 72, 72, 72],
               [15, 30, 15,  0, 72, 72, 72, 72],
               [72, 72, 72, 72,  0, 15, 15, 30],
               [72, 72, 72, 72, 15,  0, 30, 15],
               [72, 72, 72, 72, 15, 30,  0, 15],
               [72, 72, 72, 72, 30, 15, 15,  0]]

# [[60, [{0, 1}, {0, 1}, {2, 3}, {2, 3}, {4, 5}, {4, 5}, {6, 7}, {6, 7}]]]

topo_matrix_1 = [[0, 30, 15, 15, 72, 72, 72, 72],
                [30,  0, 15, 15, 72, 72, 72, 72],
                [15, 15,  0, 30, 72, 72, 72, 72],
                [15, 15, 30,  0, 72, 72, 72, 72],
                [72, 72, 72, 72,  0, 15, 15, 30],
                [72, 72, 72, 72, 15,  0, 30, 15],
                [72, 72, 72, 72, 15, 30,  0, 15],
                [72, 72, 72, 72, 30, 15, 15,  0]]

# [[90, [{0, 1}, {0, 1}, {2, 3}, {2, 3}, {4, 5}, {4, 5}, {6, 7}, {6, 7}]]]


topo_matrix_2 = [[ 0, 72, 72, 72, 72, 72, 72, 72],
                 [ 72, 0, 72, 72, 72, 72, 72, 72],
                 [ 72, 72, 0, 72, 72, 72, 72, 72],
                 [ 72, 72, 72, 0, 72, 72, 72, 72],
                 [ 72, 72, 72, 72, 0, 72, 72, 72],
                 [ 72, 72, 72, 72, 72, 0, 72, 72],
                 [ 72, 72, 72, 72, 72, 72, 0, 72],
                 [ 72, 72, 72, 72, 72, 72, 72, 0]]
MP = 2

# 0 [[0, 30, 15, 15, 
# 1 [30,  0, 15, 15, 
# 2 [15, 15,  0, 30, 
# 3 [15, 15, 30,  0,



In [ ]:
############## How if (N % MP) != 0? ############## TODO
############## How if (N % MP) != 0? ############## TODO
############## How if (N % MP) != 0? ############## TODO: Add one more layer? N = 4  but we have 8 GPUs 
"""
(0, 1, 2, 3)
(0, 1, 2, 4)
(0, 1, 2, 5)
(0, 1, 2, 6)
.
.
.
C8_4?
"""

def procesResults(candidate):
  candidate = list(candidate)
  for i, ch in enumerate(candidate):
    candidate[i] = ord(candidate[i]) - ord('a')
  return candidate


def findBestSubgroup(subgroup, topo_matrix):
  candidates = list(it.permutations(subgroup, 4))
  minSum = float('inf')
  minCom = ""
  for candidate in candidates:
    tempSum = 0
    for i in range(len(candidate) - 1):
      tempSum += topo_matrix[ord(candidate[i]) - ord('a')][ord(candidate[i + 1]) - ord('a')]
    if tempSum < minSum:
      minSum = tempSum
      # minCom = candidate
      minCom = procesResults(candidate)
  return minSum, minCom

def findBestGroup(result, topo_matrix, N, MP):
  minSum_group = float('inf')
  minCom_group = []
  for group in result: # ('ab', 'cd', 'ef', 'gh')
    tempSum = 0
    tempCom = []
    for subgroup in group: # 'ab'
      minSum, minCom = findBestSubgroup(subgroup, topo_matrix)
      tempSum += minSum
      tempCom.append(minCom)
    if tempSum <= minSum_group:  ## Shall we collect the possibility with the same minimum sum of link weights?
      if tempSum == minSum_group:
        minCom_group.append([tempSum, tempCom])
      else:
        minSum_group = tempSum
        minCom_group = [[tempSum, tempCom]]
      # print(tempCom)
  return minCom_group

In [ ]:
N = 8
MP = 4
minGroups = findBestGroup(result, topo_matrix_2, N, MP)
# print(minSum)
minGroups

In [2]:
import sys
sys.path.append("/opt/rocm/bin")
import rocm_smi


In [6]:
rocm_smi?

Type:        module
String form: <module 'rocm_smi' from '/opt/rocm/bin/rocm_smi.py'>
File:        /opt/rocm/bin/rocm_smi.py
Docstring:  
ROCm_SMI_LIB CLI Tool

This tool acts as a command line interface for manipulating
and monitoring the amdgpu kernel, and is intended to replace
and deprecate the existing rocm_smi.py CLI tool.
It uses Ctypes to call the rocm_smi_lib API.
Recommended: At least one AMD GPU with ROCm driver installed
Required: ROCm SMI library installed (librocm_smi64)


In [ ]:
if args.device is not None:
        deviceList = []
        for device in args.device:
            if not doesDeviceExist(device):
                logging.warning('No such device card%s', str(device))
                sys.exit()
            if (isAmdDevice(device) or args.alldevices) and device not in deviceList:
                deviceList.append(device)
    else:
        deviceList = rocm_smi.listDevices()

In [14]:
deviceList = rocm_smi.listDevices()

def listDevices():
    """ Returns a list of GPU devices """
    numberOfDevices = c_uint32(0)
    ret = rocmsmi.rsmi_num_monitor_devices(byref(numberOfDevices))
    if rocm_smi.rsmi_ret_ok(ret):
        deviceList = list(range(numberOfDevices.value))
        return deviceList
    else:
        exit(ret)


In [35]:
import sys
sys.path.append("/opt/rocm/bin")
import rocm_smi
from rsmiBindings import *
def listDevices():
    """ Returns a list of GPU devices """
    numberOfDevices = c_uint32(0)
    ret = rocmsmi.rsmi_num_monitor_devices(byref(numberOfDevices))
    if rocm_smi.rsmi_ret_ok(ret):
        deviceList = list(range(numberOfDevices.value))
        print("====", deviceList, numberOfDevices)
        return deviceList
    else:
        exit(ret)
        
if __name__ == '__main__':
    listDevices()

==== [] c_uint(0)


In [36]:
listDevices()



==== [] c_uint(0)


[]

In [37]:
# weight = c_uint64()
ret = rocmsmi.rsmi_topo_get_link_weight(srcdevice, destdevice, byref(weight))

NameError: name 'srcdevice' is not defined

In [26]:
rocm_smi.showWeightTopology([0,1,2,3,4,5,6,7])

ERROR:root:RSMI_STATUS_INVALID_ARGS: The provided arguments do not meet the preconditions required for calling this function.	


=========================== Weight between two GPUs ============================


TypeError: printErrLog() takes 2 positional arguments but 3 were given

In [22]:
!rocm-smi --showtoponuma



======================= ROCm System Management Interface =======================
================================== Numa Nodes ==================================
GPU[0]		: (Topology) Numa Node: 0
GPU[0]		: (Topology) Numa Affinity: 0
GPU[1]		: (Topology) Numa Node: 0
GPU[1]		: (Topology) Numa Affinity: 0
GPU[2]		: (Topology) Numa Node: 0
GPU[2]		: (Topology) Numa Affinity: 0
GPU[3]		: (Topology) Numa Node: 0
GPU[3]		: (Topology) Numa Affinity: 0
GPU[4]		: (Topology) Numa Node: 1
GPU[4]		: (Topology) Numa Affinity: 1
GPU[5]		: (Topology) Numa Node: 1
GPU[5]		: (Topology) Numa Affinity: 1
GPU[6]		: (Topology) Numa Node: 1
GPU[6]		: (Topology) Numa Affinity: 1
GPU[7]		: (Topology) Numa Node: 1
GPU[7]		: (Topology) Numa Affinity: 1
============================= End of ROCm SMI Log ==============================


In [48]:
import sys
sys.path.append("/opt/rocm/bin")
import rocm_smi
import numpy as np
from rsmiBindings import *
rocm_smi.initializeRsmi()
deviceList = rocm_smi.listDevices()
devices_ind = range(len(deviceList))
link_weights = np.zeros(shape=(len(devices_ind),len(devices_ind)), dtype=np.uint32)
numa_nodes = np.zeros(shape=(len(devices_ind)), dtype=np.uint32)

# rocm_smi.showWeightTopology(deviceList)
def showWeightTopology(deviceList, link_weights):
    # gpu_links_weight = [[0 for x in devices_ind] for y in devices_ind]
    for srcdevice in deviceList:
        for destdevice in deviceList:
            if (srcdevice == destdevice):
                # gpu_links_weight[srcdevice][destdevice] = 0
                link_weights[srcdevice][destdevice] = 0
                continue
            weight = c_uint64()
            ret = rocmsmi.rsmi_topo_get_link_weight(srcdevice, destdevice, byref(weight))
            if rocm_smi.rsmi_ret_ok(ret):
                # gpu_links_weight[srcdevice][destdevice] = weight.value
                link_weights[srcdevice][destdevice] = weight.value
            else:
                rocm_smi.printErrLog(srcdevice, 'Cannot read Link Weight: Not supported on this machine', None)
    # return gpu_links_weight
    return link_weights

# rocm_smi.showWeightTopology(deviceList)
def showNumaTopology(deviceList, numa_nodes):
    """ Display the HW Topology Information for numa nodes
    This reads the HW Topology file and display the matrix for the nodes
    @param deviceList: List of DRM devices (can be a single-item list)
    """
    numa_numbers = c_uint32()
    for device in deviceList:
        ret = rocmsmi.rsmi_topo_get_numa_node_number(device, byref(numa_numbers))
        if rocm_smi.rsmi_ret_ok(ret, device):
            numa_nodes[device] = numa_numbers.value
        else:
            rocm_smi.printErrLog(device, "Cannot read Numa Node")
    return numa_nodes

In [34]:
link_weights = showWeightTopology(deviceList, link_weights)

In [49]:
numa_nodes = showNumaTopology(deviceList, numa_nodes)

int

---
# The device ids of ROCm-SMI are as same as those of HIP.


In [20]:
len(gpu_links_weight)
for src, link_list in enumerate(gpu_links_weight):
    for des, weight in enumerate(link_list):
        if src == des: print(weight)
        else: print(weight.value)

0
15
30
15
72
72
72
72
15
0
15
30
72
72
72
72
30
15
0
15
72
72
72
72
15
30
15
0
72
72
72
72
72
72
72
72
0
15
15
30
72
72
72
72
15
0
30
15
72
72
72
72
15
30
0
15
72
72
72
72
30
15
15
0


In [5]:
def showWeightTopology(deviceList):
    """ Display the HW Topology Information based on weights
    This reads the HW Topology file and displays the matrix for the nodes
    @param deviceList: List of DRM devices (can be a single-item list)
    """
    global PRINT_JSON
    devices_ind = range(len(deviceList))
    gpu_links_weight = [[0 for x in devices_ind] for y in devices_ind]
    printLogSpacer(' Weight between two GPUs ')
    for srcdevice in deviceList:
        for destdevice in deviceList:
            if (srcdevice == destdevice):
                gpu_links_weight[srcdevice][destdevice] = 0
                continue
            weight = c_uint64()
            ret = rocmsmi.rsmi_topo_get_link_weight(srcdevice, destdevice, byref(weight))
            if rsmi_ret_ok(ret):
                gpu_links_weight[srcdevice][destdevice] = weight
            else:
                printErrLog(srcdevice, 'Cannot read Link Weight: Not supported on this machine', None)

    if PRINT_JSON:
        formatMatrixToJSON(deviceList, gpu_links_weight, "(Topology) Weight between DRM devices {} and {}")
        return

    printTableRow(None, '      ')
    for row in deviceList:
        tmp = 'GPU%d' % row
        printTableRow('%-12s', tmp)
    printEmptyLine()
    for gpu1 in deviceList:
        tmp = 'GPU%d' % gpu1
        printTableRow('%-6s', tmp)
        for gpu2 in deviceList:
            if (gpu1 == gpu2):
                printTableRow('%-12s', '0')
            else:
                printTableRow('%-12s', gpu_links_weight[gpu1][gpu2].value)
        printEmptyLine()

[0, 1, 2, 3, 4, 5, 6, 7]

In [53]:
!/opt/rocm/bin/rocm-smi --showtopoweight --json > rocm_smi.json

In [ ]:
python /opt/rocm/bin/rocm_smi.py --showtopoweight